In [ ]:
#main
import numpy as np
import pandas as pd

In [ ]:
# support
import corelib

# csv to pd loader
from corelib.data_loader import loader

# reduce memory usage for pd-index
from corelib.data_prep import reduce_mem_usage
from corelib.data_prep import reduce_obj_mem_usage

# searching for unical ordered values
from corelib.data_prep import search_func

#dump data with
from corelib.data_dump import dumper

### dump from this point

In [ ]:
print(loader.__doc__)

In [ ]:
loader(mode='tree')

In [ ]:
loader(mode='view')

In [ ]:
data = loader(mode='extract')

In [ ]:
data_seq1 = loader(mode='extract', data_for_load={'sample.csv':{}})

In [ ]:
data_for_load = {'samplezip.zip': 
                 {'index_col': 'family_id', 
                  'dtype': {'assigned_day': np.float64}},
                'anydata.csv':
                {'parse_dates': ['timestamp']}}
data_seq2 = loader(mode='extract', data_for_load=data_for_load)

In [ ]:
data_seq2['anydata.csv'].head(1)

In [ ]:
df_train, df_test = data.values()
del data

In [ ]:
# reduce memory usage
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

In [ ]:
data = df_train, df_test

In [ ]:
# Dump loaded and prepared data
dumper(dump_list=data, path='loaded_data', method='shelve', task='s')

### to this point

In [ ]:
# Dump open (prepared dataset)
df_train, df_test = dumper(path='loaded_data', method='shelve', task='o')